In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import os

import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import torchvision
import torchvision.transforms as transforms

matplotlib.rcParams['figure.figsize'] = (20,10)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
#Det har jeg ikke fået til at virke endnu

cpu


## Indlæs data

In [3]:
data_raw = np.load('../data/cullpdb+profile_5926.npy')

## Omform data

In [4]:
data = data_raw.reshape((-1,700,57))

data.shape
x = data[:,:,:22]
y = data[:,:,22:31] # (brug 35 hvis du vil have solvent properties med)
print('Fuldt datasæt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

#x = x.reshape(-1,22,700)
#y = y.reshape(-1,9,700)
x = np.rot90(x, axes=(1,2))
y = np.rot90(y, axes=(1,2))

x = np.flip(x, 1)
y = np.flip(y, 1)

print('Fuldt datasæt vendt shape:')
print('X: ', x.shape)
print('Y: ', y.shape)

x_train = x[:5430]
y_train = y[:5430]

x_test = x[5435:5690]
y_test = y[5435:5690]

x_validation = x[5690:5926]
y_validation = y[5690:5926]

print('Splittet ud i training og testing:')
print('(Train) X: ', x_train.shape)
print('(Train) Y: ', y_train.shape)
print('(Test)  X: ', x_test.shape)
print('(Test)  Y: ', y_test.shape)
print('(Validation)  X: ', x_validation.shape)
print('(Validation)  Y: ', y_validation.shape)

torch_X_train = torch.from_numpy(x_train).type(torch.FloatTensor)
torch_Y_train = torch.from_numpy(y_train).type(torch.FloatTensor)
torch_X_test  = torch.from_numpy(x_test).type(torch.FloatTensor)
torch_Y_test  = torch.from_numpy(y_test).type(torch.FloatTensor)
torch_X_validation  = torch.from_numpy(x_validation).type(torch.FloatTensor)
torch_Y_validation  = torch.from_numpy(y_validation).type(torch.FloatTensor)

Fuldt datasæt shape:
X:  (5926, 700, 22)
Y:  (5926, 700, 9)
Fuldt datasæt vendt shape:
X:  (5926, 22, 700)
Y:  (5926, 9, 700)
Splittet ud i training og testing:
(Train) X:  (5430, 22, 700)
(Train) Y:  (5430, 9, 700)
(Test)  X:  (255, 22, 700)
(Test)  Y:  (255, 9, 700)
(Validation)  X:  (236, 22, 700)
(Validation)  Y:  (236, 9, 700)


## Sæt data sammen i DataLoader

In [5]:
# Data Loader for easy mini-batch return in training, the image batch shape will be (50, 1, 28, 28)
#train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

BATCH_SIZE = 250

train = torch.utils.data.TensorDataset(torch_X_train, torch_Y_train)
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=False)

## Definér modellen

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (700, 22)
            nn.Conv1d(
                in_channels=22,            # input height
                out_channels=30,           # n_filters / feature maps
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (300, 22)
            nn.ReLU(),                      # activation
        )
        self.conv2 = nn.Sequential(         # input shape (700, 22)
            nn.Conv1d(
                in_channels=30,            # input height
                out_channels=20,           # n_filters / feature maps
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (300, 22)
            nn.ReLU(),                      # activation
        )
        self.out = nn.Sequential(         # input shape (700, 22)
            nn.Conv1d(
                in_channels=20,            # input height
                out_channels=9,           # n_filters / feature maps
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (300, 22)
            nn.ReLU(),                      # activation
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        output = self.out(x)
        return output, x    # return x for visualization

## Instantiér modellen

In [7]:
cnn = CNN()
print(cnn) # net architecture

CNN(
  (conv1): Sequential(
    (0): Conv1d(22, 30, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
  )
  (conv2): Sequential(
    (0): Conv1d(30, 20, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
  )
  (out): Sequential(
    (0): Conv1d(20, 9, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): Softmax()
  )
)


## Hyperparametre

In [11]:
EPOCH = 5               # train the training data n times, to save time, we just train 1 epoch
LR = 0.005              # learning rate

## Instantiér loss-funktioner og optimizer

In [9]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.BCELoss()  #Binary Cross Entropy Loss
validation_loss_func = nn.BCELoss()

## Træn modellen

In [255]:
# Det her virker heller ikke endnu 
# - det skal bruges til at omforme vores forudsigelser til at kunne sammenligne det med vores valideringssæt
def RemakeOneHot(a):
    #a = np.array([[0, 1], [2, 3], [4, 5], [6, 7], [9, 8]])
    b = np.zeros_like(a)
    #return (a == a.max(axis=1)[:,None]).astype(int)
    return (b[np.arange(len(a)), a.argmax(1)] == 1)

In [12]:
for epoch in range(EPOCH):
    print('Epoch: ', epoch+1)
    for step, (b_x, b_y) in enumerate(train_loader):

        output = cnn(b_x)[0]               # cnn output
        #print(output.shape)
        #print(b_y.shape)
        #print(output[0,:,0])
        #print(b_y[0,:,0])
        loss = loss_func(output, b_y)   # cross entropy loss
        #print(loss)
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 10 == 0:
            #print(loss)
            test_output, last_layer = cnn(torch_X_validation)
            #print(test_output.shape)
            vloss = validation_loss_func(test_output, torch_Y_validation)
            print(vloss)
#            print(RemakeOneHot(test_output[0].detach().numpy()))
            #accuracy = float((test_output.detach().numpy() == y_validation.data).astype(int).sum()) / float(y_validation.size(0))
            #break
            #print(loss)
            #print(pred_y[0])
            #accuracy = float((pred_y == y_test.data).astype(int).sum()) / float(test_y.size(0))
            #print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
            

Epoch:  1
tensor(0.3351, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3319, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3298, grad_fn=<BinaryCrossEntropyBackward>)
Epoch:  2
tensor(0.3293, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2050, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0799, grad_fn=<BinaryCrossEntropyBackward>)
Epoch:  3
tensor(0.0798, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0781, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0777, grad_fn=<BinaryCrossEntropyBackward>)
Epoch:  4
tensor(0.0774, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0771, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0768, grad_fn=<BinaryCrossEntropyBackward>)
Epoch:  5
tensor(0.0767, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0763, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0757, grad_fn=<BinaryCrossEntropyBackward>)


In [11]:
# Den oprindelige kode jeg stjal - bare som backup

for epoch in range(EPOCH):
    for step, b_x in enumerate(torch_X_train):   # gives batch data, normalize x when iterate train_loader
        b_y = torch_Y_train[step]
        output = cnn(b_x)#[0]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 50 == 0:
            test_output, last_layer = cnn(torch_X_test)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == torch_Y_test.data.numpy()).astype(int).sum()) / float(torch_Y_test.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [10, 22, 5], but got 2-dimensional input of size [700, 22] instead